In [16]:
import pandas as pd
import joblib
import sys
import os
from sklearn.metrics import classification_report


project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.data.preprocessor import TabnetPreprocessor
from pytorch_tabnet.tab_model import TabNetClassifier


In [17]:
MODEL_SIZE = 'small' 
CLASSIFICATION = 'multiclass'

OUTPUT_DIR = '../src/model' 

MODEL_PATH = os.path.join(OUTPUT_DIR, f'tabnet_{CLASSIFICATION}_model_{MODEL_SIZE}.zip')
PREPROCESSOR_PATH = os.path.join(OUTPUT_DIR, f'preprocessor_{CLASSIFICATION}_{MODEL_SIZE}.pkl')
ENCODER_PATH = os.path.join(OUTPUT_DIR,f'label_encoder_{CLASSIFICATION}_{MODEL_SIZE}.pkl')
TEST_SET_PATH = '../src/data/test_set.csv' 
TARGET_COLUMN = 'Attack' 

print(f"Model path: {MODEL_PATH}")
print(f"Preprocessor path: {PREPROCESSOR_PATH}")
print(f"Test set path: {TEST_SET_PATH}")

Model path: ../src/model/tabnet_multiclass_model_small.zip
Preprocessor path: ../src/model/preprocessor_multiclass_small.pkl
Test set path: ../src/data/test_set.csv


In [18]:
print("Model and preprocessor loading...")

try:
    loaded_preprocessor = joblib.load(PREPROCESSOR_PATH)

    loaded_model = TabNetClassifier()
    loaded_model.load_model(MODEL_PATH)

    print("Loaded")

except FileNotFoundError as e:
    print(f"ERROR: File not found. {e}")


Model and preprocessor loading...
Loaded


/home/mat-rib/Documents/neural-tabular-intrusion-detection-system/venv/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [19]:
print("Test set and label ancoder loading")
le = joblib.load(ENCODER_PATH)

test_df = pd.read_csv(TEST_SET_PATH)

X_test = test_df.drop(columns=[TARGET_COLUMN])
y_test = test_df[TARGET_COLUMN]
y_test_encoded = le.transform(y_test)

print(f": {len(test_df)} rows.")

X_test_processed = loaded_preprocessor.transform(X_test)


Test set and label ancoder loading
: 448668 rows.


/home/mat-rib/Documents/neural-tabular-intrusion-detection-system/src/data/preprocessor.py:131: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[col_name].fillna(unknown_code, inplace=True)
/home/mat-rib/Documents/neural-tabular-intrusion-detection-system/src/data/preprocessor.py:133: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on

In [20]:
print("Prediciting...")
y_pred = loaded_model.predict(X_test_processed.values)

y_pred_labels = le.inverse_transform(y_pred)
y_test_labels = le.inverse_transform(y_test_encoded)

report = classification_report(y_test_labels, y_pred_labels)

print("\n==============================================")
print(f"  Report (Model: {CLASSIFICATION} {MODEL_SIZE})")
print("==============================================")
print(report)

Prediciting...

  Report (Model: multiclass small)
                precision    recall  f1-score   support

      Analysis       0.21      0.98      0.35       245
      Backdoor       0.17      0.10      0.12       692
        Benign       1.00      1.00      1.00    430217
           DoS       0.15      0.18      0.16      1016
      Exploits       0.90      0.48      0.63      7768
       Fuzzers       0.66      0.86      0.75      5132
       Generic       0.25      0.60      0.35       980
Reconnaissance       0.63      0.58      0.60      2265
     Shellcode       0.40      0.66      0.49       328
         Worms       0.42      0.60      0.49        25

      accuracy                           0.98    448668
     macro avg       0.48      0.60      0.49    448668
  weighted avg       0.99      0.98      0.98    448668

